In [18]:
import warnings
warnings.filterwarnings("ignore")

In [19]:
import pandas as pd

clients = pd.read_csv('../input/clients.csv', parse_dates=["joined"])
loans = pd.read_csv('../input/loans.csv', parse_dates=["loan_start", "loan_end"])
payments = pd.read_csv('../input/payments.csv' , parse_dates=["payment_date"])

In [20]:
clients.head(3)

,client_id,joined,income,credit_score
0,46109,2002-04-16,172677,527
1,49545,2007-11-14,104564,770
2,41480,2013-03-11,122607,585


In [21]:
loans.head(3)

,client_id,loan_type,loan_amount,repaid,loan_id,loan_start,loan_end,rate
0,46109,home,13672,0,10243,2002-04-16,2003-12-20,2.15
1,46109,credit,9794,0,10984,2003-10-21,2005-07-17,1.25
2,46109,home,12734,1,10990,2006-02-01,2007-07-05,0.68


In [22]:
payments.head(3)

,loan_id,payment_amount,payment_date,missed
0,10243,2369,2002-05-31,1
1,10243,2439,2002-06-18,1
2,10243,2662,2002-06-29,0


In [23]:
import numpy as np
clients["join_month"] = clients["joined"].dt.month
clients["log_income"] = np.log(clients["income"])
clients.head(3)

,client_id,joined,income,credit_score,join_month,log_income
0,46109,2002-04-16,172677,527,4,12.059178
1,49545,2007-11-14,104564,770,11,11.557555
2,41480,2013-03-11,122607,585,3,11.716739


In [24]:
stats = loans.groupby("client_id")["loan_amount"].agg(["mean", "max", "min"])
stats.columns = ["mean_loan_amount", "max_loan_amount", "min_loan_amount"]
stats.head(3)

,mean_loan_amount,max_loan_amount,min_loan_amount
client_id,,,
25707,7963.950000,13913,1212
26326,7270.062500,13464,1164
26695,7824.722222,14865,2389


In [25]:
clients_stats_merge = pd.merge(
    clients,
    stats,
    left_on="client_id",
    right_on="client_id",
    how="left",
)

clients_stats_merge.head(3)

,client_id,joined,income,credit_score,join_month,log_income,mean_loan_amount,max_loan_amount,min_loan_amount
0,46109,2002-04-16,172677,527,4,12.059178,8951.60,14049,559
1,49545,2007-11-14,104564,770,11,11.557555,10289.30,14971,3851
2,41480,2013-03-11,122607,585,3,11.716739,7894.85,14399,811


In [26]:
clients_stats_loans_merge = pd.merge(
    clients_stats_merge,
    loans,
    left_on="client_id",
    right_on="client_id",
    how="left",
)

clients_stats_loans_merge.head(3)

,client_id,joined,income,credit_score,join_month,log_income,mean_loan_amount,max_loan_amount,min_loan_amount,loan_type,loan_amount,repaid,loan_id,loan_start,loan_end,rate
0,46109,2002-04-16,172677,527,4,12.059178,8951.6,14049,559,home,13672,0,10243,2002-04-16,2003-12-20,2.15
1,46109,2002-04-16,172677,527,4,12.059178,8951.6,14049,559,credit,9794,0,10984,2003-10-21,2005-07-17,1.25
2,46109,2002-04-16,172677,527,4,12.059178,8951.6,14049,559,home,12734,1,10990,2006-02-01,2007-07-05,0.68


In [27]:
merge_all = pd.merge(
    clients_stats_loans_merge,
    payments,
    left_on="loan_id",
    right_on="loan_id",
    how="left",
)

merge_all.head(3)

,client_id,joined,income,credit_score,join_month,log_income,mean_loan_amount,max_loan_amount,min_loan_amount,loan_type,loan_amount,repaid,loan_id,loan_start,loan_end,rate,payment_amount,payment_date,missed
0,46109,2002-04-16,172677,527,4,12.059178,8951.6,14049,559,home,13672,0,10243,2002-04-16,2003-12-20,2.15,2369,2002-05-31,1
1,46109,2002-04-16,172677,527,4,12.059178,8951.6,14049,559,home,13672,0,10243,2002-04-16,2003-12-20,2.15,2439,2002-06-18,1
2,46109,2002-04-16,172677,527,4,12.059178,8951.6,14049,559,home,13672,0,10243,2002-04-16,2003-12-20,2.15,2662,2002-06-29,0


In [28]:
stats_payment = merge_all.groupby("loan_id")["payment_amount"].agg(
    ["sum", "mean", "max", "min", "std", "var"]
)

stats_payment.columns = [
    "sum_payment_amount",
    "mean_payment_amount",
    "max_payment_amount",
    "min_payment_amount",
    "std_payment_amount",
    "var_payment_amount",
]

stats_payment.head(3)

,sum_payment_amount,mean_payment_amount,max_payment_amount,min_payment_amount,std_payment_amount,var_payment_amount
loan_id,,,,,,
10009,1996,332.666667,394,280,46.915527,2201.066667
10017,9432,1179.000000,1446,994,155.324545,24125.714286
10024,6345,705.000000,895,518,141.412517,19997.500000


In [29]:
merge_all_add_newft = pd.merge(
    merge_all,
    stats_payment,
    left_on="loan_id",
    right_on="loan_id",
    how="left",
)

merge_all_add_newft.head(3)

,client_id,joined,income,credit_score,join_month,log_income,mean_loan_amount,max_loan_amount,min_loan_amount,loan_type,...,rate,payment_amount,payment_date,missed,sum_payment_amount,mean_payment_amount,max_payment_amount,min_payment_amount,std_payment_amount,var_payment_amount
0,46109,2002-04-16,172677,527,4,12.059178,8951.6,14049,559,home,...,2.15,2369,2002-05-31,1,14008,2334.666667,2662,2027,212.940054,45343.466667
1,46109,2002-04-16,172677,527,4,12.059178,8951.6,14049,559,home,...,2.15,2439,2002-06-18,1,14008,2334.666667,2662,2027,212.940054,45343.466667
2,46109,2002-04-16,172677,527,4,12.059178,8951.6,14049,559,home,...,2.15,2662,2002-06-29,0,14008,2334.666667,2662,2027,212.940054,45343.466667


In [30]:
import featuretools as ft

In [31]:
es = ft.EntitySet(id="clients")

In [32]:
es = es.entity_from_dataframe(
    entity_id="clients",
    dataframe=clients,
    index="client_id",
    time_index="joined",
)

In [34]:
es = es.entity_from_dataframe(
    entity_id="loans",
    dataframe=loans,
    variable_types={"repaid": ft.variable_types.Categorical},
    index="loan_id",
    time_index="loan_start",
)

In [35]:
es = es.entity_from_dataframe(
    entity_id="payments",
    dataframe=payments,
    variable_types={"missed": ft.variable_types.Categorical},
    make_index=True,
    index="payment_id",
    time_index="payment_date",
)

In [36]:
es

Entityset: clients
  Entities:
    clients [Rows: 25, Columns: 6]
    loans [Rows: 443, Columns: 8]
    payments [Rows: 3456, Columns: 5]
  Relationships:
    No relationships

In [39]:
es["clients"]

Entity: clients
  Variables:
    client_id (dtype: index)
    joined (dtype: datetime_time_index)
    income (dtype: numeric)
    credit_score (dtype: numeric)
    join_month (dtype: numeric)
    log_income (dtype: numeric)
  Shape:
    (Rows: 25, Columns: 6)

In [37]:
es["loans"]

Entity: loans
  Variables:
    loan_id (dtype: index)
    client_id (dtype: numeric)
    loan_type (dtype: categorical)
    loan_amount (dtype: numeric)
    loan_start (dtype: datetime_time_index)
    loan_end (dtype: datetime)
    rate (dtype: numeric)
    repaid (dtype: categorical)
  Shape:
    (Rows: 443, Columns: 8)

In [38]:
es["payments"]

Entity: payments
  Variables:
    payment_id (dtype: index)
    loan_id (dtype: numeric)
    payment_amount (dtype: numeric)
    payment_date (dtype: datetime_time_index)
    missed (dtype: categorical)
  Shape:
    (Rows: 3456, Columns: 5)

In [40]:
relation_client_loans = ft.Relationship(
    es["clients"]["client_id"],
    es["loans"]["client_id"],
)

es = es.add_relationship(relation_client_loans)

In [41]:
relation_loans_payments = ft.Relationship(
    es["loans"]["loan_id"],
    es["payments"]["loan_id"],
)

es = es.add_relationship(relation_loans_payments)

In [48]:
primitives = ft.list_primitives()
primitives[primitives["type"] == "aggregation"].head(10)

,name,type,dask_compatible,koalas_compatible,description
0,median,aggregation,False,False,Determines the middlemost number in a list of ...
1,avg_time_between,aggregation,False,False,Computes the average number of seconds between...
2,sum,aggregation,True,True,"Calculates the total addition, ignoring `NaN`."
3,num_true,aggregation,True,False,Counts the number of `True` values.
4,trend,aggregation,False,False,Calculates the trend of a variable over time.
5,max,aggregation,True,True,"Calculates the highest value, ignoring `NaN` v..."
6,time_since_last,aggregation,False,False,Calculates the time elapsed since the last dat...
7,any,aggregation,True,False,Determines if any value is 'True' in a list.
8,mode,aggregation,False,False,Determines the most commonly repeated value.
9,count,aggregation,True,True,"Determines the total number of values, excludi..."


In [49]:
primitives[primitives["type"] == "transform"].head(10)

,name,type,dask_compatible,koalas_compatible,description
22,hour,transform,True,True,Determines the hour value of a datetime.
23,cum_min,transform,False,False,Calculates the cumulative minimum.
24,age,transform,True,False,Calculates the age in years as a floating poin...
25,divide_by_feature,transform,True,True,Divide a scalar by each value in the list.
26,cum_count,transform,False,False,Calculates the cumulative count.
27,scalar_subtract_numeric_feature,transform,True,True,Subtract each value in the list from a given s...
28,haversine,transform,False,False,Calculates the approximate haversine distance ...
29,num_words,transform,True,True,Determines the number of words in a string by ...
30,absolute,transform,True,True,Computes the absolute value of a number.
31,add_numeric_scalar,transform,True,True,Add a scalar to each value in the list.


In [50]:
features, feature_names = ft.dfs(
    entityset = es, 
    target_entity = 'clients', 
    agg_primitives = ['mean', 'max', 'percent_true', 'last'],
    trans_primitives = ['years', 'month', 'divide']
)

ValueError: ('Unknown transform primitive years. ', 'Call ft.primitives.list_primitives() to get', ' a list of available primitives')

In [ ]:
pd.DataFrame(features['MONTH(joined)'].head())

In [ ]:
pd.DataFrame(features['MEAN(payments.payment_amount)'].head())

In [ ]:
features.head()

In [ ]:
pd.DataFrame(features['MEAN(loans.loan_amount)'].head(10))

In [ ]:
pd.DataFrame(features['LAST(loans.MEAN(payments.payment_amount))'].head(10))

In [ ]:
features, feature_names = ft.dfs(entityset=es, target_entity='clients', 
                                 max_depth = 2)

In [ ]:
features.iloc[:, 4:].head()